Esta es una primer prueba a entender el problema para el equipo de text mining. 

Considerar que todo lo redactado puede tener errores de redaccion, tipograficos, etc. porque le estoy prestando cero atencion a eso. Solo intento desglozar algunas partes del problema.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Una de las cosas mas dificiles de esta competencia es el propio dataset. El cual no existe y tenemos que construirlo nosotros de manera "creativa" usando otros data sets anteriores de competencias similares. 

A priori parece lo que lo mas inteligente es usar dos datasets (Del segundo no estoy tan convencido aun) Que son  [jigsaw-multilingual-toxic-comment-classification](http://../input/jigsaw-toxic-comment-classification-challenge) y [ruddit jigsaw dataset](http://../input/ruddit-jigsaw-dataset).

A priori voy a usar el primero para crear un dataset y ver si finalmente necesitamos algo del segundo. Super importante entender que como creemos este dataset va a impactar de manera directa en que tan buenas sean finalmente nuestras decisiones. 

In [ ]:
def count_upper_case_letters(str_obj):
    count = 0
    for elem in str_obj:
        if elem.isupper():
            count += 1
    return count

**Creamos data set**

In [ ]:
#El data set a usar tiene filas con comentarios y columnas con categorias binarias. 
# Toxico, muy toxico, obseno, amenaza, insulto, discrimacion hacia la identidad, contenido sexual explicito

#ponderador
ponderador_de_toxicidad = {
    'toxic': 1,
    'severe_toxic': 2,
    'obscene': 1,
    'threat': 1,
    'insult': 1,
    'identity_hate': 2,
    'sexual_explicit': 1
}

toxicity_types = list(ponderador_de_toxicidad.keys())
toxicity_types

**Como paso siguiente deberiamos limpiar el data set**

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

pd.set_option('display.max_colwidth',300)
# Importo el dataset
df = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv', encoding='utf-8')
df.shape

In [ ]:
df.head(10)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_n_words(corpus, n=10):
    cv = CountVectorizer()
    corpus_matrix = cv.fit_transform(corpus)
    corpus_matrix = pd.DataFrame.sparse.from_spmatrix(corpus_matrix, columns=cv.get_feature_names())
    aux = corpus_matrix.sum()
    aux=aux.to_frame('count')
    aux.sort_values(by='count',ascending=False,inplace=True)
    aux=aux.head(n)
    return aux

In [ ]:
# Cantidad de tokens en todo el corpus
import plotly.express as px

most_importants = get_top_n_words(df.comment_text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

In [ ]:
df_toxic = df[(df.toxic == 1) | (df.severe_toxic == 1) ]
df_toxic

In [ ]:
# Cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df_toxic.comment_text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

Hasta aquí, hemos descargado el dataset con el que vamos a trabajar y rankeamos las principales palabras, tanto en la totalidad del dataset como en los comentarios que a priori estan definidas cómo tóxicas o super tóxicas.

Se observa que gran parte de las palabras, en ambos dataset (df y df_toxic) no tienen impacto directo sobre la definicion de si el comentario es o no tóxico. En los proximos pasos, eliminaremos las stop_words, con lo cual parte de ellas deberian desaparecer. Sin embargo, habrá palabras que sigan siendo irrelevantes para el caso particuar. Sobre estas podemos realizar un analisis particular.

**Proceso NLP**

Definimos la función con la que eliminaremos las stop_words. También tenemos la posibilidad de agregar palabras adicionales.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
stop_words = set( stopwords.words('english'))
stop_words.update(string.punctuation)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

# Pre procesamiento basico, agregar las etapas que considere necesario
def pre_procesamiento_texto(text):
  # Armo los tokens para procesar los datos
  tokens = word_tokenize(text)

  # Elimino las stopwords
  tokens = [t.lower() for t in tokens if t.lower() not in stop_words]

  return tokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Tengo que crear el dataset limpio. Es decir solo con una columna de "resultado" de si es toxico o no para poder seguir probando

In [ ]:
# Creamos un df1 para diferenciar el primer proceso (regresión) del segundo (clasificacion)
df1 = df
# Combine all toxicity levels into one with the same weights set
df1['toxicity'] = sum([df1[type] * coef for type, coef in ponderador_de_toxicidad.items() if type in df1])

# Standardize toxicity (converts to continues values)
df1['toxicity'] = df1['toxicity']/df1['toxicity'].max()

In [ ]:
# Downsample
df1 = pd.concat([
    df1[df1['toxicity'] <= 0].sample(n = int((df1['toxicity'] > 0).sum() * 1.5),random_state = 201),
    df1[df1['toxicity'] > 0]
])
print(f"- New shape: {df1.shape}")
df1

In [ ]:
import collections
df1["rep_char"] = 0
df1["rep_cap"] = 0
df1["char_len"] = 0
for row in df1.iterrows():
    string = str(df1.loc[row[0],"comment_text"])
    df1.loc[row[0],"rep_char"] = collections.Counter(string).most_common(1)[0][1]/len(string)
    df1.loc[row[0],"rep_cap"] = count_upper_case_letters(string)/len(string)
    df1.loc[row[0],"char_len"]  = len(string)
df1.head(5)

In [ ]:
import scipy as sp
# Construimos una df auxiliar para poder agregar los features "externos" al texto en al sparse matrix (matria vectorizada palabra por palabra)
df_aux = df1[["rep_char","rep_cap","char_len"]].to_numpy()
# Vectorizacion de resultados BOW y TF-IDF
cv = CountVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_cv = cv.fit_transform(df1.comment_text) # Armo matriz para entrenar CV
X_transform_cv = sp.sparse.hstack((X_transform_cv, df_aux)) # Agrego las columnas numericas
# TF-IDF 
tf = TfidfVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_tf = tf.fit_transform(df1.comment_text) # Armo matriz para entrenar tf
X_transform_tf = sp.sparse.hstack((X_transform_tf, df_aux)) # Agrego las columnas numericas
print(X_transform_cv.shape)
print(X_transform_tf.shape)

In [ ]:
#Separado X,y en train y test, dos veces porque cada proceso de NLP es distinto

X_traincv, X_testcv, y_train, y_test = train_test_split(X_transform_cv, df1.toxicity,test_size=.30)
X_traintf, X_testtf, y_train, y_test = train_test_split(X_transform_tf, df1.toxicity,test_size=.30)
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


In [ ]:
list_col = cv.get_feature_names()
list_col.append("rep_char")
list_col.append("rep_cap")
list_col.append("char_len")
df_transform = pd.DataFrame.sparse.from_spmatrix(X_traincv, columns=list_col)

In [ ]:
df_transform.head(5)

In [ ]:
aux = df_transform.drop(['rep_char','rep_cap','char_len'], axis=1)
aux = aux.sum().to_frame('count')
aux.sort_values(by='count',ascending=False,inplace=True)
aux=aux.head(100)
print(aux.head(10))
px.bar(aux, x=aux.index, y='count')

**MODELOS**

Se entrenan distintos modelos y para las distintas vectorizaciones planteadas originalmente

In [ ]:
## CV

# Entreno CV - regresión lineal
from sklearn.linear_model import LinearRegression
learner1 = LinearRegression()
learner1.fit(X_traincv, y_train)

In [ ]:
## CV

#Entreno CV - Random Forest
from sklearn.ensemble import RandomForestRegressor
learner2 = RandomForestRegressor(n_estimators = 100, random_state = 0)
learner2.fit(X_traincv, y_train)

In [ ]:
## TF

# Entreno TF - Regresion
from sklearn.linear_model import LinearRegression
learner3 = LinearRegression()
learner3.fit(X_traintf, y_train)

In [ ]:
## TF

# Entreno TF - Random Forest
from sklearn.linear_model import LinearRegression
learner4 = RandomForestRegressor(n_estimators = 100, random_state = 0)
learner4.fit(X_traintf, y_train)

In [ ]:
## CV

#Entreno CV - lightgbm
import lightgbm as lgb
learner5 = lgb.LGBMRegressor(n_estimators=1000, max_depth=10)
learner5.fit(X_traincv.astype('float32'), y_train)

In [ ]:
## TF

#Entreno CV - lightgbm
import lightgbm as lgb
learner6 = lgb.LGBMRegressor(n_estimators=1000, max_depth=10)
learner6.fit(X_traintf, y_train)

In [ ]:
#Hago predicciones sobre Test

## Para CV Regresion
y_pred1 = learner1.predict(X_testcv)

## Para CV RF
y_pred2 = learner2.predict(X_testcv)

## Para TF Regresion
y_pred3 = learner3.predict(X_testtf)

## Para TF RF
y_pred4 = learner4.predict(X_testtf)

## Para CV LGBM
y_pred5 = learner5.predict(X_testcv.astype('float32'))

## Para TF LGBM
y_pred6 = learner6.predict(X_testtf)

In [ ]:
# RMSE (Root Mean Square Error)

from sklearn.metrics import mean_squared_error 

rmse1 = float(format(np.sqrt(mean_squared_error(y_test, y_pred1)),'.3f'))
print("\nRMSE - CV Reg Mult:\n",rmse1)
rmse2 = float(format(np.sqrt(mean_squared_error(y_test, y_pred2)),'.3f'))
print("\nRMSE - CV Random Forest:\n",rmse2)
rmse3 = float(format(np.sqrt(mean_squared_error(y_test, y_pred3)),'.3f'))
print("\nRMSE - TF Reg Mult:\n",rmse3)
rmse4 = float(format(np.sqrt(mean_squared_error(y_test, y_pred4)),'.3f'))
print("\nRMSE - TF Random Forest:\n",rmse4)
rmse5 = float(format(np.sqrt(mean_squared_error(y_test, y_pred5)),'.3f'))
print("\nRMSE - TF LGBM + CV:\n",rmse5)
rmse6 = float(format(np.sqrt(mean_squared_error(y_test, y_pred6)),'.3f'))
print("\nRMSE - TF LGBM + TF:\n",rmse6)

In [ ]:
list_pred = [["1 (CV+RM)",y_pred1],["2 (CV+RF)",y_pred2],["3 (TF+RM)",y_pred3],["4 (TF+RF)",y_pred4],["5 (TF+LGBM)",y_pred5],["6 (CV+LGBM)",y_pred6]]
for idx,i in list_pred:
    aux1 = pd.DataFrame(i)
    aux2 = pd.DataFrame(y_test).reset_index()
    aux3 = aux2
    aux3['pred'] = aux1
    ax1 = aux3.plot.scatter(x = 'index', y = 'pred', color='b',title="y_test Vs. y_pred"+idx)
    ax2 = aux3.plot.scatter(x = 'index', y = 'toxicity', color='y', ax=ax1)
    print(ax1==ax2) 

**----------------------- SEGUNDA PRUEBA: Categorías de Toxicidad (c/ modelos de clasificación) -----------------------**

In [ ]:
df2 = df
# Combine all toxicity levels into one with the same weights set
df2['toxicity'] = sum([df2[type] * coef for type, coef in ponderador_de_toxicidad.items() if type in df2])

# Standardize toxicity (converts to continues values)
df2['toxicity'] = np.where(df2['toxicity']<4, 0,1)
                          #np.where(df2['toxicity']<6, 0.5, 1))

In [ ]:
# Downsample
df2 = pd.concat([
    df2[df2['toxicity'] <= 0].sample(n = int((df2['toxicity'] > 0).sum() * 1.5),random_state = 201),
    df2[df2['toxicity'] > 0]
])
print(f"- New shape: {df2.shape}")
df2

In [ ]:
import collections
df2["rep_char"] = 0
df2["rep_cap"] = 0
df2["char_len"] = 0
for row in df2.iterrows():
    string = str(df2.loc[row[0],"comment_text"])
    df2.loc[row[0],"rep_char"] = collections.Counter(string).most_common(1)[0][1]/len(string)
    df2.loc[row[0],"rep_cap"] = count_upper_case_letters(string)/len(string)
    df2.loc[row[0],"char_len"]  = len(string)
df2.head(5)

In [ ]:
df_group = df2[["toxicity", "rep_char","rep_cap","char_len"]]
df_group = df_group.groupby('toxicity').mean()
df_group

In [ ]:
import scipy as sp

# Construimos una df auxiliar para poder agregar los features "externos" al texto en al sparse matrix (matria vectorizada palabra por palabra)
df_aux = df2[["rep_char","rep_cap","char_len"]].to_numpy()

# Vectorizacion de resultados BOW y TF-IDF
cv = CountVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_cv = cv.fit_transform(df2.comment_text) # Armo matriz para entrenar CV
X_transform_cv = sp.sparse.hstack((X_transform_cv, df_aux)) # Agrego las columnas numericas

# TF-IDF 
tf = TfidfVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_tf = tf.fit_transform(df2.comment_text) # Armo matriz para entrenar tf
X_transform_tf = sp.sparse.hstack((X_transform_tf, df_aux)) # Agrego las columnas numericas

print(X_transform_cv.shape)
print(X_transform_tf.shape)

In [ ]:
#Separado X,y en train y test, dos veces porque cada proceso de NLP es distinto

X_traincv, X_testcv, y_train, y_test = train_test_split(X_transform_cv, df2.toxicity,test_size=.30)
X_traintf, X_testtf, y_train, y_test = train_test_split(X_transform_tf, df2.toxicity,test_size=.30)
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
list_col = cv.get_feature_names()
list_col.append("rep_char")
list_col.append("rep_cap")
list_col.append("char_len")
df_transform = pd.DataFrame.sparse.from_spmatrix(X_traincv, columns=list_col)

In [ ]:
df_transform.head()

**ANÁLISIS DESCRIPTIVO **

In [ ]:
import matplotlib.pyplot as plt
import string
stop_words2 = set( stopwords.words('english'))
stop_words2.add("article")
stop_words2.add("page")
stop_words2.add("'m")
stop_words2.add("'ll")
stop_words2.add("'s")
stop_words2.add("'re")
stop_words2.update(string.punctuation)

# Pre procesamiento basico, agregar las etapas que considere necesario
def pre_procesamiento_texto2(text):
  # Armo los tokens para procesar los datos
  tokens = word_tokenize(text)

  # Elimino las stopwords
  tokens = [t.lower() for t in tokens if t.lower() not in stop_words2]

  return tokens


from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Construimos una df auxiliar para poder agregar los features "externos" al texto en al sparse matrix (matria vectorizada palabra por palabra)
df_aux2 = df2[["toxicity"]].to_numpy()
df2_pos = df2[df2.toxicity == 0]
df2_neg = df2[df2.toxicity == 1]

# CV POS
cv2 = CountVectorizer(tokenizer=pre_procesamiento_texto2, max_df=0.2, min_df=0.03)
X_transform_cv2 = cv2.fit_transform(df2_pos.comment_text) # Armo matriz para entrenar CV
X_traincv2, X_testcv2, y_train2, y_test2 = train_test_split(X_transform_cv2, df2_pos.toxicity,test_size=.30)
list_col2 = cv2.get_feature_names()
df_transform_pos = pd.DataFrame.sparse.from_spmatrix(X_traincv2, columns=list_col2)
# CV NEG
cv2 = CountVectorizer(tokenizer=pre_procesamiento_texto2, max_df=0.2, min_df=0.03)
X_transform_cv2 = cv2.fit_transform(df2_neg.comment_text) # Armo matriz para entrenar CV
X_traincv2, X_testcv2, y_train2, y_test2 = train_test_split(X_transform_cv2, df2_neg.toxicity,test_size=.30)
list_col2 = cv2.get_feature_names()
print(list_col2)
df_transform_neg = pd.DataFrame.sparse.from_spmatrix(X_traincv2, columns=list_col2)

aux_pos = " ".join(df_transform_pos)
aux_neg = " ".join(df_transform_neg)

#generating the wordcloud - POS
wordcloud = WordCloud(background_color="white").generate(aux_pos)

#plot the wordcloud
plt.figure(figsize = (12, 12))
plt.imshow(wordcloud)

#to remove the axis value
plt.axis("off")
plt.show()

#generating the wordcloud - NEG
wordcloud = WordCloud(background_color="white").generate(aux_neg)

#plot the wordcloud
plt.figure(figsize = (12, 12))
plt.imshow(wordcloud)

#to remove the axis value
plt.axis("off")
plt.show()

**MODELOS**

In [ ]:
## CV

# Entreno CV - regresión lineal
from sklearn.linear_model import LogisticRegression
learner1 = LogisticRegression(solver='lbfgs', max_iter=1000)
learner1.fit(X_traincv, y_train)

In [ ]:
## CV

#Entreno CV - Random Forest
from sklearn.ensemble import RandomForestClassifier
learner2 = RandomForestClassifier(n_estimators = 100, random_state = 0)
learner2.fit(X_traincv, y_train)

In [ ]:
## TF

# Entreno TF - Regresion
from sklearn.linear_model import LogisticRegression
learner3 = LogisticRegression(solver='lbfgs', max_iter=1000)
learner3.fit(X_traintf, y_train)

In [ ]:
## TF

#Entreno CV - Random Forest
from sklearn.ensemble import RandomForestClassifier
learner4 = RandomForestClassifier(n_estimators = 100, random_state = 0)
learner4.fit(X_traintf, y_train)

In [ ]:
## CV

#Entreno CV - lightgbm
import lightgbm as lgb
learner5 = lgb.LGBMClassifier(n_estimators=1000, max_depth=10)
learner5.fit(X_traincv.astype('float32'), y_train)

In [ ]:
## TF

#Entreno CV - lightgbm
import lightgbm as lgb
learner6 = lgb.LGBMClassifier(n_estimators=1000, max_depth=10)
learner6.fit(X_traintf, y_train)

In [ ]:
#Hago predicciones sobre Test

## Para CV Regresion
y_pred1 = learner1.predict(X_testcv)

## Para CV RF
y_pred2 = learner2.predict(X_testcv)

## Para TF Regresion
y_pred3 = learner3.predict(X_testtf)

## Para TF RF
y_pred4 = learner4.predict(X_testtf)

## Para CV LGBM
y_pred5 = learner5.predict(X_testcv.astype('float32'))

## Para TF LGBM
y_pred6 = learner6.predict(X_testtf)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
list_pred = [["1 (CV+RM)",y_pred1],["2 (CV+RF)",y_pred2],["3 (TF+RM)",y_pred3],["4 (TF+RF)",y_pred4],["5 (CV+LGBM)",y_pred5],["6 (TF+LGBM)",y_pred6]]
for idx,i in list_pred:
    aux1 = pd.DataFrame(i)
    aux2 = pd.DataFrame(y_test).reset_index()
    aux3 = aux2
    aux3['pred'] = aux1
    conf_mat_aux = confusion_matrix(aux3['toxicity'], aux3['pred'])
    conf_mat = ConfusionMatrixDisplay(confusion_matrix=conf_mat_aux)
    target_names = ['No Tóxico (0)', 'Tóxico (1)']
    clas_sum = classification_report(aux3['toxicity'], aux3['pred'], target_names=target_names)
    conf_mat.plot()
    plt.title("y_test Vs. y_pred"+idx)
    plt.show()
    print(clas_sum)
    print("Precision = True Positive / (True Positive + False Positive) - (Or Negative)")
    print("Recall = True Positive / (Positive) - (Or Negative)")
    print("Deberíamos aspirar a un precision y recall altos en la categoría tóxico (1)")
    print("")